In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def make_detailed_df():
    url = 'https://www.rew.ca/properties/areas/vancouver-bc/sort/latest/asc/page/1?property_type=apartment-condo&query=Vancouver%2C+BC'
    url_pg2 = 'https://www.rew.ca/properties/areas/vancouver-bc/type/apartment-condo/sort/latest/desc/page/2?query=Vancouver%2C+BC'
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    houses = soup.find_all(lambda tag: tag.name == 'article' and 
                                       tag.get('class') == ['displaypanel'])
    prices = []
    ttl_squ_feet = []
    maintance = []
    gross_taxes = []
    bedrooms = []
    bathrooms = []
    property_type = []
    year_built = []
    styles = []
    title_list = []
    features_list = []
    amenities_list = []
    appliances_list = []
    neighborhood_list = []
    hyperlink = []
    for house in houses: 
        house_href = house.div.div.a['href']
        house_url = requests.get(f'https://www.rew.ca/{house_href}').text
        soup = BeautifulSoup(house_url,'lxml')
        
        # If there are any listing on the page which was removed add this to ignore those pages
        # or else there will be an error. 
#         main_content = soup.find('div', class_='container')
#         if str(main_content.h1)!='None':
#             continue

        ul = soup.find_all('ul',class_='listingheader-details l-pipedlist l-pipedlist--breathe')
        area = ul[0].find_all('li')[2].text
        house_details = soup.find_all('tbody')
        price_details = house_details[0].find_all('tr')
        list_price = price_details[0].td.text
        if 'Gross Taxes' not in str(price_details):
            gross_t = 'NaN'
        else: gross_t = price_details[1].td.text
        if 'Maintenance Fees' not in str(price_details):
            maintenance_fees = 'NaN'
        elif 'Gross Taxes' not in str(price_details): 
            maintenance_fees = price_details[1].td.text.replace('\n','')
        else: maintenance_fees = price_details[2].td.text.replace('\n','')
        home_facts = house_details[1].find_all('tr')
        bdrms = home_facts[0].td.text
        bthrms = home_facts[1].td.text
        ppty_type = home_facts[2].td.text
        yr_built = home_facts[3].td.text.replace('Built in ','')
        title = home_facts[4].td.text
        style = home_facts[5].td.text
        
        headers = soup.find_all('th')
        if 'Features' in str(headers):
            features = home_facts[6].td.text
            if 'Amenities' in str(headers):
                amenities = home_facts[7].td.text
                if 'Appliances' in str(headers):
                    appliances = home_facts[8].td.text
                    if 'Community' in str(headers):
                        neighborhood = home_facts[9].td.text
                    else: neighborhood = 'NaN'
                elif 'Community' in str(headers):
                    neighborhood = home_facts[8].td.text
                    appliances = 'NaN'
            elif 'Appliances' in str(headers):
                appliances = home_facts[7].td.text
                amenities = 'NaN'
                if 'Community' in str(headers):
                    neighborhood = home_facts[8].td.text
                else: neighborhood = 'NaN'
            elif 'Community' in str(headers):
                appliances = 'NaN'
                neighborhood = home_facts[7].td.text
            else: 
                appliances = 'NaN'
                neighborhood = 'NaN'
                amenities = 'NaN'
        elif 'Amenities' in str(headers):
            features = 'NaN'
            amenities = home_facts[6].td.text
            if 'Appliances' in str(headers):
                appliances = home_facts[7].td.text
                if 'Community' in str(headers):
                    neighborhood = home_facts[8].td.text
                else: 
                    neighborhood = 'NaN'
            elif 'Community' in str(headers):
                appliances = 'NaN'
                neighborhood = home_facts[7].td.text
            else: 
                neighborhood = 'NaN'
                appliances = 'NaN'
        elif 'Appliances' in str(headers):
            amenities = 'NaN'
            features = 'NaN'
            appliances = home_facts[6].td.text
            if 'Community' in str(headers):
                neighborhood = home_facts[7].td.text
            else: neighborhood = 'NaN'
        elif 'Community' in str(headers):
            amenities = 'NaN'
            features = 'NaN'
            appliances = 'NaN'
            neighborhood = home_facts[6].td.text
        else: 
            amenities = 'NaN'
            features = 'NaN'
            appliances = 'NaN'
            neighborhood = 'NaN'
            
        prices.append(list_price)
        ttl_squ_feet.append(area)
        maintance.append(maintenance_fees)
        gross_taxes.append(gross_t)
        bedrooms.append(bdrms)
        bathrooms.append(bthrms)
        property_type.append(ppty_type)
        year_built.append(yr_built)
        styles.append(style)
        amenities_list.append(amenities)
        appliances_list.append(appliances)
        neighborhood_list.append(neighborhood)
        title_list.append(title)
        hyperlink.append(f'https://www.rew.ca/{house_href}')

    my_columns = ['List Price', 
                  'Total Square Feet',
                  'Maintenance Fees', 
                  'Gross Tax', 
                  'Bedrooms', 
                  'Bathrooms',
                  'Property Type', 
                  'Year Built', 
                  'Title', 
                  'Style', 
                  'Amenities',
                  'Appliances',
                  'Neighborhood', 
                  'Link'
                 ]
    list_of_str = list(zip(prices,
                           ttl_squ_feet,
                           maintance,
                           gross_taxes,
                           bedrooms,
                           bathrooms,
                           property_type,
                           year_built,
                           title_list,
                           styles,
                           amenities_list,
                           appliances_list,
                           neighborhood_list,
                           hyperlink
                          ))
    df = pd.DataFrame(list_of_str,columns=my_columns)
    return df  
print('Setup Complete')


Setup Complete


In [18]:
dataframe = make_detailed_df()
dataframe

,List Price,Total Square Feet,Maintenance Fees,Gross Tax,Bedrooms,Bathrooms,Property Type,Year Built,Title,Style,Amenities,Appliances,Neighborhood,Link
0,"$2,099,999",788 Sqft,$437,$0,1,1,Apt/Condo,2022 (0 yrs old),Freehold Strata,Upper Unit,"Air Conditioning, Central Location",Washer/Dryer/Fridge/Stove/Dishwasher,West End,https://www.rew.ca//properties/R2312605/1005-1...
1,"$1,749,888",723 Sqft,$457,$0,0,1,Apt/Condo,2022 (0 yrs old),Freehold Strata,Upper Unit,"Air Conditioning, Central Location, Shopping N...",Washer/Dryer/Fridge/Stove/Dishwasher,West End,https://www.rew.ca//properties/R2312567/402-15...
2,"$2,899,000",1335 Sqft,$496,$0,3,2,Apt/Condo,2019 (3 yrs old),Leasehold not prepaid-NonStrata,"Corner Unit,End Unit,Upper Unit","Golf Course Nearby, Greenbelt, Recreation Near...","Clothes Dryer, Clothes Washer, Dishwasher, Mic...",University (UBC),https://www.rew.ca//properties/R2377913/1808-3...
3,"$1,799,000",670 Sqft,$423,$0,1,1,Apt/Condo,2022 (0 yrs old),Freehold Strata,Other,Air Conditioning,Washer/Dryer/Fridge/Stove/Dishwasher,West End,https://www.rew.ca//properties/R2422318/1102-1...
4,"$4,352,888",1347 Sqft,$853,$0,2,2,Apt/Condo,2021 (1 yrs old),Freehold Strata,Other,Air Conditioning,Washer/Dryer/Fridge/Stove/Dishwasher,West End,https://www.rew.ca//properties/R2422507/1402-1...
5,"$1,259,900",935 Sqft,$461,"$2,646",3,2,Apt/Condo,2014 (8 yrs old),Freehold Strata,1 Storey,"Air Conditioning, Recreation Nearby, Shopping ...","Clothes Dryer, Clothes Washer, Compactor - Gar...",Southwest Marine,https://www.rew.ca//properties/2712566/501-848...
6,"$1,690,000",858 Sqft,$627,"$5,219",1,2,Apt/Condo,2016 (6 yrs old),Freehold Strata,1 Storey,NaN,NaN,Coal Harbour,https://www.rew.ca//properties/2762081/5901-11...
7,"$1,399,900",1175 Sqft,$598,"$2,869",2,2,Apt/Condo,1994 (28 yrs old),Freehold Strata,5 Plus Level,"Private Setting, Shopping Nearby, Treed",Washer/Dryer/Fridge/Stove/Dishwasher,Yaletown,https://www.rew.ca//properties/2816271/602-283...
8,"$5,498,000",1422 Sqft,NaN,$0,2,2,Apt/Condo,2021 (1 yrs old),Freehold Strata,Upper Unit,"Air Conditioning, Central Location, Marina Nea...",Washer/Dryer/Fridge/Stove/Dishwasher,West End,https://www.rew.ca//properties/2952014/3402-15...
9,"$409,000",590 Sqft,$604,$0,1,1,Apt/Condo,1968 (54 yrs old),Leasehold prepaid-NonStrata,1 Storey,"Central Location, Recreation Nearby, Shopping ...","Refrigerator, Stove",West End,https://www.rew.ca//properties/3067593/1008-18...


In [19]:
def save_df(df,filename):
    df.to_csv(fr'/Users/veliristimaki/Downloads/data_scaping/{filename}.csv', index = False)
    
save_df(dataframe,'housing_pg1')